In [93]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py
!pip install matplotlib

from bbmodcache.bbSearch import SearchProblem, search
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy
from enum import Enum

Installing bbSearch module from web ...
creating bbmodcache subfolder


A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file bbmodcache already exists.
Error occurred while processing: bbmodcache.


downloading bbSearch module


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 18767  100 18767    0     0   238k      0 --:--:-- --:--:-- --:--:--  251k


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# B - Robot Worker

In [94]:
class Item(Enum):
    RUSTY_KEY = "Rusty Key"
    BUCKET = "Bucket"
    SUITCASE = "Suitcase"
    SCREWDRIVER = "Screwdriver"
    SLEDGE_HAMMER = "Sledge Hammer"
    ANVIL = "Anvil"
    SAW = "Saw"
    #GOLD_BAR = "Gold Bar"
    #SILVER_BAR = "Silver Bar"
    #DIAMOND_RING = "Diamond Ring"
    #OVEN_MITS = "Oven Mits"
    #HOT_SOUP = "Hot Soup"
    #LASAGNA = "Lasagna"

class Room(Enum):
    WORKSHOP = "Workshop"
    STORE_ROOM = "Store Room"
    TOOL_CUPBOARD = "Tool Cupboard"
    #KITCHEN = "Kitchen"
    #BEDROOM = "Bedroom"


In [95]:
class Robot:
    def __init__(self, location : Room, carried_items : list[Item], strength : int):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHTS[i] for i in self.carried_items])
    
    def value_carried(self):
        return sum([ITEM_VALUES[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA : Room, roomB : Room, doorkey : Item = None, locked : bool =False):
        self.goes_between = { roomA, roomB }
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [96]:
class State:
    def __init__(self, robot : Robot, doors : list[Door], room_contents : dict[Room, set[Item]]):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    # Calculates the value of a room's contents
    def room_value(self, room : Room) -> int:
        return sum(ITEM_VALUES[i] for i in self.room_contents[room])

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )

In [97]:
class GoalParam:
    """
    Specifies a set of items that are required to be in a room
    and the minimum value of all the items in the room
    """
    
    def __init__(self, room : Room, contents : set[Item], desired_value : int = 0):
        self.room = room
        self.contents = contents
        self.desired_value = desired_value

In [98]:
class Action(Enum):
    CHANGE_ROOM = "Change room to"
    PICK_UP = "Pick up"
    PUT_DOWN = "Put down"

class RobotWorker( SearchProblem ):
    def __init__( self, state : State, goal_params : set[GoalParam] ):
        self.initial_state = state
        self.goal_params = goal_params

    def possible_actions( self, state : State ):
        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions : list[tuple[Action, Item | Room]] = []
        # Can put down any carried item
        for item in state.robot.carried_items:
            # If item has dependents
            if item in DEPENDENT_ITEMS:
                dependent_items = DEPENDENT_ITEMS[item]
                # Check if any dependent item is in inventory
                valid = True
                for dependent in dependent_items:
                    if dependent in state.robot.carried_items:
                        valid = False
                        break
                if valid:
                    actions.append((Action.PUT_DOWN, item))
            else:
                actions.append((Action.PUT_DOWN, item))


        # Can pick up any item in room if strong enough and has item it depends on
        for item in state.room_contents[robot_location]:
            if strength >= weight_carried + ITEM_WEIGHTS[item]:
                # If item has required items
                if item in REQUIRED_ITEMS:
                    for required_item in REQUIRED_ITEMS[item]:
                        if required_item in state.robot.carried_items:
                            actions.append((Action.PICK_UP, item))
                else:
                    actions.append((Action.PICK_UP, item))


        # If there is an unlocked door between robot location and
        # another location can move to that location
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append((Action.CHANGE_ROOM, door.other_loc[robot_location]))

        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state : State, action : tuple[Action, Item | Room]):
        next_state = deepcopy(state)
        act, target = action
        if act == Action.PUT_DOWN:
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == Action.PICK_UP:
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == Action.CHANGE_ROOM:
            next_state.robot.location = target

        return next_state

    def goal_test(self, state : State):
        #print(state.room_contents)
        for goal_param in self.goal_params:
            room = goal_param.room
            contents = goal_param.contents
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
                    
            # Check desired_value is greater (or equal) to the value in the room
            desired_value = goal_param.desired_value
            actual_value = state.room_value(room)
            if desired_value < actual_value:
                return False

        return True

    def display_state(self, state : State):
        print("Robot location:", state.robot.location.value)
        print("Robot carrying:", [item.value for item in state.robot.carried_items])
        print("Room contents:", [(room.value, [item.value for item in items]) for room, items in state.room_contents.items()])
        print("Room values:", [(room.value, state.room_value(room)) for room in state.room_contents])

In [99]:
ROOM_CONTENTS = {
    Room.WORKSHOP : { Item.RUSTY_KEY},#, Item.GOLD_BAR, Item.SILVER_BAR, Item.OVEN_MITS },
    Room.STORE_ROOM : { Item.BUCKET, Item.SUITCASE },
    Room.TOOL_CUPBOARD : { Item.SLEDGE_HAMMER, Item.ANVIL, Item.SAW, Item.SCREWDRIVER}#, Item.DIAMOND_RING },
    #Room.KITCHEN : { Item.HOT_SOUP, Item.LASAGNA },
    #Room.BEDROOM : set()
}

ITEM_WEIGHTS = {
    Item.RUSTY_KEY : 0,
    Item.BUCKET : 2,
    Item.SUITCASE : 4,
    Item.SCREWDRIVER : 1,
    Item.SLEDGE_HAMMER : 5,
    Item.ANVIL : 12,
    Item.SAW : 2,
    #Item.GOLD_BAR : 7,
    #Item.SILVER_BAR : 3,
    #Item.DIAMOND_RING : 1,
    #Item.OVEN_MITS : 0,
    #Item.HOT_SOUP : 3,
    #Item.LASAGNA : 3
}

ITEM_VALUES = {
    Item.RUSTY_KEY : -10,
    Item.BUCKET : -2,
    Item.SUITCASE : 1,
    Item.SCREWDRIVER : 2,
    Item.SLEDGE_HAMMER : 5,
    Item.ANVIL : 16,
    Item.SAW : -1,
    #Item.GOLD_BAR : 35,
    #Item.SILVER_BAR : 19,
    #Item.DIAMOND_RING : 43,
    #Item.OVEN_MITS : 0,
    #Item.HOT_SOUP : 2,
    #Item.LASAGNA : 6
}

DOORS = [
    Door(Room.WORKSHOP, Room.STORE_ROOM),
    Door(Room.STORE_ROOM, Room.TOOL_CUPBOARD, doorkey=Item.RUSTY_KEY, locked=False),
    #Door(Room.KITCHEN, Room.WORKSHOP),
    #Door(Room.KITCHEN, Room.BEDROOM),
    #Door(Room.BEDROOM, Room.WORKSHOP)
]

# Helper function
def reverse_dependency_map(dependent_items):
    reverse_map = {}

    for item, dependents in dependent_items.items():
        for dependent in dependents:
            if dependent not in reverse_map:
                reverse_map[dependent] = set()
            reverse_map[dependent].add(item)

    return reverse_map

# One item in set must be in inventory before item can be picked up
# Item cannot be removed if any items in set still being carried
DEPENDENT_ITEMS = {
    #Item.OVEN_MITS : {Item.HOT_SOUP, Item.LASAGNA},
    #Item.HOT_SOUP : {Item.GOLD_BAR}
}

REQUIRED_ITEMS = reverse_dependency_map(DEPENDENT_ITEMS)

In [101]:
robot = Robot(Room.STORE_ROOM, [], 30)
initial_state = State(robot, DOORS, ROOM_CONTENTS)
goal_params = {
    #GoalParam(Room.BEDROOM, {Item.DIAMOND_RING}, 50),
    #GoalParam(Room.KITCHEN, {Item.OVEN_MITS}),
    GoalParam(Room.STORE_ROOM, {Item.SLEDGE_HAMMER,Item.SCREWDRIVER, Item.ANVIL})
}

RW_PROBLEM_1 = RobotWorker(initial_state, goal_params)

In [103]:
def heu(state : State):
    return state.room_value(state.robot.location) + state.robot.value_carried()

search(RW_PROBLEM_1, 'BF/FIFO', 100000, loop_check=True)#, heuristic=heu)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.........................................................
!! Search node limit (100000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   354581  (includes start)
Nodes discarded by loop_check  =   254580  (100001 distinct states added to queue)
Nodes tested (by goal_test)    =    57769  (all expanded)
Nodes left in queue            =    42231

Time taken = 28.4057 seconds



'NODE_LIMIT_EXCEEDED'